In [1]:
import pandas as pd
from numpy import random
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.models import load_model

Using TensorFlow backend.


In [12]:
data = pd.read_csv(r"E:\EEG DATA\train_ordered\02_tcp_le\meta_final.csv")
data.head()
data.shape

(508, 7)

In [3]:
(data['stop_time'] - data['start_time']).idxmax()

231

In [4]:
maxlen = 231*250

In [13]:
#Folder to load the data from
folder = "E:\\EEG DATA\\train_ordered\\02_tcp_le\\train_ready\\"
features = []

#Making a feature matrix - Links of all CSVs to be loaded
for i in range(508):
        features.append(folder+str(i)+".csv")

In [14]:
#bianrizing the labels
def binarize(label):
    if label == 'seiz':
        return 1
    else:
        return 0
    
data['label'] = data['label'].apply(lambda lab: binarize(lab))

data['label']

0      0
1      0
2      0
3      0
4      0
5      1
6      0
7      0
8      0
9      0
10     0
11     0
12     0
13     0
14     0
15     0
16     0
17     0
18     0
19     0
20     0
21     0
22     0
23     1
24     0
25     0
26     0
27     1
28     0
29     0
      ..
478    0
479    0
480    0
481    0
482    1
483    0
484    0
485    1
486    0
487    0
488    1
489    0
490    0
491    1
492    0
493    0
494    1
495    0
496    0
497    0
498    0
499    1
500    0
501    0
502    1
503    0
504    0
505    0
506    1
507    0
Name: label, Length: 508, dtype: int64

In [32]:
def load_and_process(link):
    #Read data into a dataframe
    data = pd.read_csv(link, header=None)
    #Convert dataframe to numpy matrix for padding
    data = data.values
    #Pad sequences as per maxlen
    data = keras.preprocessing.sequence.pad_sequences(data, padding='post', maxlen=maxlen)
    return data 

# #Making a generator 
# def generator(features, labels):
#     while True:
#         #Pick a random index
#         index= random.choice(len(features),1)
#         #Load and preprocess the data for that index
#         batch_features = load_and_process(features[index[0]])
#         batch_features = np.array(batch_features).reshape((1, 32, maxlen))
#         #Get label for index
#         batch_labels = labels[index[0]].reshape(1)
#         #YEILD TO THE OVERLORDS!
#         print(batch_features.shape)
#         print(batch_labels.shape)
#         yield  (batch_features, batch_labels)
        
#With batch_size
def generator(features, labels, batch_size):
    # Create empty arrays to contain batch of features and labels#
    batch_features = np.zeros((batch_size, 32, maxlen))
    batch_labels = np.zeros((batch_size,1))
    while True:
        for i in range(batch_size):
            # choose random index in features
            index= random.choice(len(features),1)
            batch_features[i] = load_and_process(features[index[0]])
            batch_labels[i] = labels[index[0]]
        print(batch_features.shape,"|", batch_labels.shape)
        yield batch_features, batch_labels

In [33]:
# #Visualizing what the generator is actually returning
# N = generator(features, data['label'])
# print(*N, sep='\n')

In [34]:
model = Sequential()

#model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100, input_shape=(32, maxlen), recurrent_dropout=0.2))
# model.add(keras.layers.Flatten())
model.add(Dense(1, activation='softmax'))
model.compile(loss='binary_crossentropy',optimizer='sgd', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 100)               23140400  
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 101       
Total params: 23,140,501
Trainable params: 23,140,501
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit_generator(generator(features, data['label'].apply(lambda x: int(x)), 10), samples_per_epoch=10, nb_epoch=20)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  """Entry point for launching an IPython kernel.
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=10, epochs=20)`
  """Entry point for launching an IPython kernel.


Epoch 1/20
(10, 32, 57750) | (10, 1)
 6/10 [=================>............] - ETA: 10:02 - loss: 11.4254 - acc: 0.2833

In [ ]:
model.save('E:\\EEG DATA\\train_ordered\\02_tcp_le\\model1.h5')